<h2>Importing necessary modules for this project</h2>

In [2]:
from bs4 import BeautifulSoup
import requests
import time
import json

<h3>getting the html text file as response from using request</h3>

In [2]:
response = requests.get('https://en.wikipedia.org/wiki/The_Illusionist_(2010_film)')
html_text = response.text
print(html_text[:500])

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>The Illusionist (2010 film) - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"8f1c7334-4845-40cb-99b8-5


<h3>pass the text file to Beautiful soup for parsing</h3>

In [3]:
soup = BeautifulSoup(html_text, 'lxml')
type(soup)

bs4.BeautifulSoup

In [4]:
table = soup.find('table', class_='infobox vevent') # selecting the table containing all the info

<h3>Getting all the rows in the table which has two columns</h3>

In [5]:
rows = table.find_all('tr')
rows[:3]

[<tr><th class="infobox-above summary" colspan="2" style="font-size: 125%; font-style: italic;">The Illusionist</th></tr>,
 <tr><td class="infobox-image" colspan="2"><a class="image" href="/wiki/File:L%27illusionniste-poster.jpg"><img alt="L'illusionniste-poster.jpg" class="thumbborder" data-file-height="267" data-file-width="200" decoding="async" height="267" src="//upload.wikimedia.org/wikipedia/en/7/70/L%27illusionniste-poster.jpg" width="200"/></a><div class="infobox-caption">Theatrical release poster</div></td></tr>,
 <tr><td class="infobox-full-data description" colspan="2"><link href="mw-data:TemplateStyles:r1066479718" rel="mw-deduplicated-inline-style"/></td></tr>]

In [25]:
def get_row_data_values(row_datas):
    if row_datas.find('li') or row_datas.find('br'):
        return [text.replace('\xa0', '') for text in row_datas.stripped_strings 
                if ')' not in text  if '(' not in text]
    
    return row_datas.get_text(strip=True).replace('\xa0', '')

def remove_tags(row_datas):
    for sup in row_datas.find_all('sup'):
        sup.decompose()
        
    for span in row_datas.find_all('span'):
        span.decompose()
        
    return row_datas
    

In [7]:
info_table = {}
for index, row in enumerate(rows):
    if index == 0:
        info_table["Title"] = row.text
        continue
    elif index == 1:
        continue
    if not row.find('th'):
        continue
        
    row_name = row.find('th').text
    row_datas = row.find("td", class_='infobox-data')
    cln_row_datas = remove_tags(row_datas)
    
    if row_name == "Based on":
        row_data_values = cln_row_datas.get_text(',', strip=True)
        info_table[row_name] = row_data_values
        continue
        
    row_data_values = get_row_data_values(cln_row_datas)
    
    info_table[row_name] = row_data_values

info_table

{'Title': 'The Illusionist',
 'French': "L'Illusionniste",
 'Directed by': 'Sylvain Chomet',
 'Screenplay by': ['Henri Marquet', 'Sylvain Chomet'],
 'Story by': 'Jacques Tati',
 'Based on': 'The Illusionist,by Jacques Tati',
 'Produced by': ['Sally Chomet', 'Bob Last'],
 'Starring': ['Jean-Claude Donda', 'Eilidh Rankin'],
 'Edited by': 'Sylvain Chomet',
 'Music by': 'Sylvain Chomet',
 'Productioncompanies': ['Pathé', 'Django Films', 'Allied Filmmakers'],
 'Distributed by': ['Pathé Distribution', 'Warner Bros. Entertainment UK'],
 'Release dates': ['16June2010', '20August2010'],
 'Running time': '79 minutes',
 'Countries': ['France', 'United Kingdom'],
 'Languages': ['French', 'English', 'Gaelic'],
 'Budget': '$17 million',
 'Box office': '$6 million'}

<h3>Getting all the links of movie pages of <b>2010-1019</b></h3>

In [3]:
markup = requests.get('https://en.wikipedia.org/wiki/List_of_Warner_Bros._films_(2010–2019)').text
soup = BeautifulSoup(markup, 'lxml')

<h3>Selecting the movie list table</h3>

In [4]:
table_list = soup.find("table", class_='wikitable sortable')

In [18]:
movie_rows = table_list.find_all('tr')
len(movie_rows)

375

In [17]:
movie_paths = []

movie_rows = table_list.find_all('tr')

for index, movie_row in enumerate(movie_rows):
    try:
        if index == 0:
            continue

        movie_title = movie_row.find('i')
        relative_path = movie_title.find('a')['href']
        abs_path = 'https://en.wikipedia.org' + relative_path

        movie_paths.append(abs_path)

        if index%50 == 0:
            print(index)

        time.sleep(2)
        
    except:
        print('Link not found')

Link not found
Link not found
50
Link not found
100
150
Link not found
Link not found
200
Link not found
Link not found
250
Link not found
300
Link not found
350


In [19]:
len(movie_paths)

365

<h3>Lets save the paths to json file to use it later</h3>

In [20]:
with open('./files/movie_paths.json', "w", encoding='utf-8') as file:
    json.dump(movie_paths, file, ensure_ascii=False, indent=2)

<h3>lets load the paths</h3>

In [23]:
with open('./files/movie_paths.json', 'r', encoding='utf-8') as file:
    movie_paths = json.load(file)

len(movie_paths)

365

In [22]:
print(movie_paths[10])

https://en.wikipedia.org/wiki/Clash_of_the_Titans_(2010_film)


<h3>Lets get the information for each wavepage and store them as a list</h3>

In [24]:
# a function that returns the table information
def get_table(soup):
    info_table = {}
    
    table = soup.find('table', class_='infobox vevent')
    rows = table.find_all('tr')
    
    for index, row in enumerate(rows):
        if index == 0:
            info_table["Title"] = row.text
            continue
        elif index == 1:
            continue
        if not row.find('th'):
            continue

        row_name = row.find('th').text
        row_datas = row.find("td", class_='infobox-data')
        cln_row_datas = remove_tags(row_datas)

        if row_name == "Based on":
            row_data_values = cln_row_datas.get_text(',', strip=True)
            info_table[row_name] = row_data_values
            continue

        row_data_values = get_row_data_values(cln_row_datas)

        info_table[row_name] = row_data_values
        
    return info_table

In [29]:
movie_dataset = []
for index, path in enumerate(movie_paths):
    try:
        info_table = {}

        markup = requests.get(path).text
        soup = BeautifulSoup(markup, 'lxml')

        table = get_table(soup)
        title = path.split('/')[-1]

        info_table[title] = table
        movie_dataset.append(info_table)
        time.sleep(2)
        
        if index % 100 == 0:
            print(index)
            
    except:
        print(f'{index} could not scrap table for path: {path}')

0
41 could not scrap table for path: https://en.wikipedia.orghttps://it.wikipedia.org/wiki/La_bellezza_del_somaro
48 could not scrap table for path: https://en.wikipedia.orghttps://pl.wikipedia.org/wiki/Jak_si%C4%99_pozby%C4%87_cellulitu
100
103 could not scrap table for path: https://en.wikipedia.orghttps://tr.wikipedia.org/wiki/%C3%87anakkale_%C3%87ocuklar%C4%B1
106 could not scrap table for path: https://en.wikipedia.orghttps://tr.wikipedia.org/wiki/O%C4%9Flum_Bak_Git
109 could not scrap table for path: https://en.wikipedia.orghttps://tr.wikipedia.org/wiki/%C3%87akallarla_Dans_2:_Hastas%C4%B1y%C4%B1z_Dede
118 could not scrap table for path: https://en.wikipedia.orghttps://tr.wikipedia.org/wiki/Hititya:_Madalyonun_S%C4%B1rr%C4%B1
138 could not scrap table for path: https://en.wikipedia.orghttps://tr.wikipedia.org/wiki/Vay_Ba%C5%9F%C4%B1ma_Gelenler
145 could not scrap table for path: https://en.wikipedia.orghttps://tr.wikipedia.org/wiki/Kedi_%C3%96zledi
147 could not scrap table for p

In [31]:
len(movie_dataset)

335

In [32]:
movie_dataset[-50]

{'Rampage_(2018_film)': {'Title': 'Rampage',
  'Directed by': 'Brad Peyton',
  'Screenplay by': ['Ryan Engle',
   'Carlton Cuse',
   'Ryan J. Condal',
   'Adam Sztykiel'],
  'Story by': 'Ryan Engle',
  'Based on': 'Rampage,by,Midway Games',
  'Produced by': ['Dwayne Johnson',
   'Brad Peyton',
   'Beau Flynn',
   'John Rickard',
   'Hiram Garcia'],
  'Starring': ['Dwayne Johnson',
   'Naomie Harris',
   'Malin Åkerman',
   'Joe Manganiello',
   'Jake Lacy',
   'Marley Shelton',
   'Jeffrey Dean Morgan'],
  'Cinematography': 'Jaron Presant',
  'Edited by': ['Jim May', 'Bob Ducsay'],
  'Music by': 'Andrew Lockington',
  'Productioncompanies': ['New Line Cinema',
   'Access Entertainment',
   'Dune Entertainment',
   'Flynn Picture Company',
   'Wrigley Pictures',
   'ASAP Entertainment',
   'Seven Bucks Productions'],
  'Distributed by': 'Warner Bros. Pictures',
  'Release dates': ['April4,2018', 'Microsoft Theater', 'April13,2018'],
  'Running time': '107 minutes',
  'Country': 'United 

<h3>From 365 webpages we were able to scrap 335 pages. The ones which are not written in English or 
is not a movie. Now lets save the data into a json file</h3>

In [34]:
with open('./files/movie_dataset.json', "w", encoding='utf-8') as file:
    json.dump(movie_dataset, file, ensure_ascii=False, indent=4)

<h1>The data collection part is now completed right now we need to clean it</h1>
<H2><b>Check the data_cleaning notebook for cleaning the movie_dataset</b><H2>